In [1]:
# Import Modules
import isaacgym
from legged_gym import LEGGED_GYM_ROOT_DIR
from legged_gym.envs import *
from legged_gym.utils import  get_args, export_policy_as_jit, task_registry, Logger
from legged_gym.utils.helpers import class_to_dict,get_load_path, PolicyExporter
from isaacgym.torch_utils import *
from rsl_rl.modules import ActorCritic, AsymActorCritic
from ruamel.yaml import dump, RoundTripDumper
from datetime import datetime
import numpy as np
import torch
import copy
import os
import sys
sys.argv = sys.argv[0:1]+["--task=gr1t2_leg"]+["--sim_device=cuda:1"]+["--rl_device=cuda:1"]
# sys.argv.append("--headless")
sys.argc = len(sys.argv)
args = get_args()

Importing module 'gym_38' (/home/willw/isaacgym/isaacgym/python/isaacgym/_bindings/linux-x86_64/gym_38.so)
Setting GYM_USD_PLUG_INFO_PATH to /home/willw/isaacgym/isaacgym/python/isaacgym/_bindings/linux-x86_64/usd/plugInfo.json
PyTorch version 2.0.1+cu117
Device count 2
/home/willw/isaacgym/isaacgym/python/isaacgym/_bindings/src/gymtorch


Using /home/willw/.cache/torch_extensions/py38_cu117 as PyTorch extensions root...
Emitting ninja build file /home/willw/.cache/torch_extensions/py38_cu117/gymtorch/build.ninja...
Building extension module gymtorch...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


ninja: no work to do.


Loading extension module gymtorch...


Warp 0.10.1 initialized:
   CUDA Toolkit: 11.5, Driver: 12.2
   Devices:
     "cpu"    | x86_64
     "cuda:0" | NVIDIA GeForce RTX 3090 (sm_86)
     "cuda:1" | NVIDIA GeForce RTX 3090 (sm_86)
   Kernel cache: /home/willw/.cache/warp/0.10.1


In [2]:
class DreamWaqInference(torch.nn.Module):
    def __init__(self, prototype, state_dict=None):
        super(DreamWaqInference, self).__init__()
        proto_cpu = copy.deepcopy(prototype).cpu()
        if hasattr(proto_cpu,"fc_mu"):
            latent_dim = proto_cpu.fc_mu.out_features
        else:
            latent_dim = 0
        for k,l in proto_cpu.est_layers.items():
            latent_dim += l.out_features
#         for mlp
        self.ob_dims = prototype.actor[0].in_features - latent_dim
        # for gru
#         self.ob_dims = ppo_runner.alg.actor_critic.actor.rnn.weight_ih_l0.size()[1] - latent_dim
        self.encoder_dims = prototype.encoder[0].in_features
        self.encoder_components = [e for e in proto_cpu.encoder]
        
        self.encoder_components.append(torch.nn.Linear(self.encoder_components[-2].out_features, 
                                       latent_dim))
        
        self.actor = proto_cpu.actor.cpu()
        print("self.actor:", self.actor)
        self.composite_encoder = torch.nn.Sequential(*self.encoder_components).cpu()
        
        if state_dict is not None:
            encoder_params = {k.split(".",1)[-1]:d for k,d in state_dict.items() if k.startswith("encoder")}
            actor_params = {k.split(".",1)[-1]:d for k,d in state_dict.items() if k.startswith("actor")}
            num_layers = len(encoder_params)
            
            est_weights = []
            est_bias = []
            
            for k in state_dict:
                if k.startswith("est_layers."):
                    if k.endswith(".weight"):
                        est_weights.append(state_dict[k])
                    elif k.endswith(".bias"):
                        est_bias.append(state_dict[k])
            if "fc_mu.weight" in state_dict.keys():
                est_weights.append(state_dict["fc_mu.weight"])
                est_bias.append(state_dict["fc_mu.bias"])
            
            encoder_params[f"{num_layers}.weight"] = torch.cat(est_weights, dim=0)
            encoder_params[f"{num_layers}.bias"] = torch.cat(est_bias, dim=0)
            
            self.actor.load_state_dict(actor_params)
            self.composite_encoder.load_state_dict(encoder_params)
        
    def get_policy_scripts(self):
        obs = torch.rand(4, self.encoder_dims).float()
        action = self.forward(obs)
        actor_script = torch.jit.script(self.actor)
        encoder_script = torch.jit.script(self.composite_encoder)
        policy_in1_script = torch.jit.script(self)
        return encoder_script, actor_script, policy_in1_script
        
    def forward(self, obs_h):
        z = self.composite_encoder(obs_h)
        obz = torch.cat([obs_h[..., -self.ob_dims:],z], dim=-1)
        action = self.actor(obz)
        return action

In [3]:
env_cfg, train_cfg = task_registry.get_cfgs(name=args.task)
env_cfg.env.num_envs = min(env_cfg.env.num_envs, 4)
train_cfg.runner.resume = True
train_cfg.runner.load_run = "241010_134454_lower_500Hz"
train_cfg.runner.checkpoint = -1

env_cfg.terrain.curriculum = False
env_cfg.terrain.mesh_type = 'plane' 
# env_cfg.terrain.mesh_type = 'trimesh' 
# env_cfg.terrain_proportions = [0.2, 0.2, 0.2, 0.2, 0.2]
env_cfg.num_rows = 9  # number of terrain rows (levels)
env_cfg.num_cols = 9  # number of terrain cols (types)
env_cfg.noise.add_noise = False
env_cfg.domain_rand.randomize_friction = False
env_cfg.domain_rand.randomize_base_mass = False
env_cfg.domain_rand.randomize_motor_strength = False
env_cfg.domain_rand.randomize_Kp_factor = False
env_cfg.domain_rand.randomize_Kd_factor = False
env_cfg.domain_rand.push_robots = False
env_cfg.domain_rand.randomize_lag_timesteps = False
env_cfg.domain_rand.randomize_init_state = False

In [4]:
env_cfg.noise.add_noise = True
env_cfg.domain_rand.randomize_friction = True
env_cfg.domain_rand.randomize_base_mass = True
env_cfg.domain_rand.randomize_motor_strength = True
env_cfg.domain_rand.randomize_Kp_factor = True
env_cfg.domain_rand.randomize_Kd_factor = True
# env_cfg.domain_rand.push_robots = True
env_cfg.domain_rand.randomize_lag_timesteps = True
env_cfg.domain_rand.randomize_init_state = True

In [5]:
env, _ = task_registry.make_env(name=args.task, args=args, env_cfg=env_cfg)
for i in range(3):
    _ = env.reset()

Setting seed: 1730104782.4298208
Not connected to PVD
+++ Using GPU PhysX
Physics Engine: PhysX
Physics Device: cuda:1
GPU Pipeline: enabled
self.num_dof: 29 	self.num_dofs: 29
body_names: ['base_link', 'left_thigh_roll_link', 'left_thigh_yaw_link', 'left_thigh_pitch_link', 'left_shank_pitch_link', 'left_foot_pitch_link', 'left_foot_roll_link', 'right_thigh_roll_link', 'right_thigh_yaw_link', 'right_thigh_pitch_link', 'right_shank_pitch_link', 'right_foot_pitch_link', 'right_foot_roll_link', 'waist_yaw_link', 'waist_pitch_link', 'waist_roll_link', 'left_upper_arm_pitch_link', 'left_upper_arm_roll_link', 'left_upper_arm_yaw_link', 'left_lower_arm_pitch_link', 'left_hand_yaw_link', 'left_hand_roll_link', 'left_hand_pitch_link', 'right_upper_arm_pitch_link', 'right_upper_arm_roll_link', 'right_upper_arm_yaw_link', 'right_lower_arm_pitch_link', 'right_hand_yaw_link', 'right_hand_roll_link', 'right_hand_pitch_link']
dof_names: ['left_hip_roll_joint', 'left_hip_yaw_joint', 'left_hip_pitch_jo

/home/willw/anaconda3/envs/isaac/lib/python3.8/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


reward_names: ['ang_Vel', 'bHgt', 'bRot', 'bTwist', 'cotr', 'eFrc', 'eVel', 'jVel', 'lin_Vel', 'smth']


In [6]:
weight_path = get_load_path(f"{LEGGED_GYM_ROOT_DIR}/logs/{train_cfg.runner.experiment_name}", 
                         load_run=train_cfg.runner.load_run, 
                         checkpoint=train_cfg.runner.checkpoint)
iter_num = weight_path.rsplit("_")[-1][:-3]
root_dir, exp_name = weight_path.rsplit("/",2)[:2]
save_dir = os.path.join(root_dir, 'exported', f'{exp_name}.{iter_num}')

print(weight_path)
print(save_dir)

/home/willw/isaacgym/LeggedGym-317/logs/GR1T2_blind/241010_134454_lower_500Hz/model_10000.pt
/home/willw/isaacgym/LeggedGym-317/logs/GR1T2_blind/exported/241010_134454_lower_500Hz.10000


In [7]:
ppo_runner, train_cfg = task_registry.make_alg_runner(env=env, name=args.task, args=args, train_cfg=train_cfg)

'train_cfg' provided -> Ignoring 'name=gr1t2_leg'
{'activation': 'elu', 'actor_hidden_dims': [512, 256, 128], 'critic_hidden_dims': [1024, 512, 128], 'encoder_hidden_dims': [1024, 256, 64], 'estimation_dims': {'vel': 3, 'implicit': 16}, 'init_noise_std': 1.0, 'latent_dims': 19, 'rnn_hidden_size': 512, 'rnn_num_layers': 1, 'rnn_type': 'gru', 'token_channels': 64} <class 'dict'>
AsymActorCritic.__init__ got unexpected args, ignoring: ['num_heightmap_obs', 'vision_in_channels', 'latent_dims', 'rnn_hidden_size', 'rnn_num_layers', 'rnn_type', 'token_channels']
[AsymAC] Actor MLP: Sequential(
  (0): Linear(in_features=68, out_features=512, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ELU(alpha=1.0)
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ELU(alpha=1.0)
  (6): Linear(in_features=128, out_features=12, bias=True)
)
[AsymAC] Critic MLP: Sequential(
  (0): Linear(in_features=80, out_features=1024, bias=True)
  (1): 

In [8]:
ep = DreamWaqInference(ppo_runner.alg.actor_critic, 
                      state_dict=torch.load(weight_path, map_location=env.device)['model_state_dict'])
en_sc, ac_sc, p1_sc = ep.get_policy_scripts()

self.actor: Sequential(
  (0): Linear(in_features=68, out_features=512, bias=True)
  (1): ELU(alpha=1.0)
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ELU(alpha=1.0)
  (4): Linear(in_features=256, out_features=128, bias=True)
  (5): ELU(alpha=1.0)
  (6): Linear(in_features=128, out_features=12, bias=True)
)


In [9]:
# Auxillary data preparation
stds = env_cfg.normalization.obs_scales

acMargin = np.array([[-10,10]] * env_cfg.env.num_actions)
actMean = np.array([env_cfg.init_state.default_joint_angles[j] for j in env_cfg.normalization.whole_body_dof_order])

actStd = np.array([env_cfg.control.action_scale[j.rsplit("_",1)[0].split("_",1)[1]] for j in env_cfg.normalization.action_dof_order])
print("actStd:",actStd)
obsMean = np.zeros(env_cfg.env.num_observations)
obsStd = np.array([1/stds.ang_vel]*3+
                  [-1]*3+
                  [1/stds.lin_vel]*2+
                  [1/stds.ang_vel]+
                  [1/stds.dof_pos]*len(env_cfg.normalization.obs_dof_order)+
                  [1/stds.dof_vel]*len(env_cfg.normalization.obs_dof_order)+
                  [1]*actStd.shape[0]+
                  [1]*4)
print(obsStd.shape)
obsStd[9:21] = actStd

whole_body_keys = []
for j in env_cfg.normalization.whole_body_dof_order:
    if j.startswith("left") or j.startswith("right") :
        whole_body_keys.append(j.rsplit("_",1)[0].split("_",1)[1])
    else:
        whole_body_keys.append(j.rsplit("_",1)[0])
print(whole_body_keys)

pGain = np.array([env_cfg.control.stiffness[j] for j in whole_body_keys])
dGain = np.array([env_cfg.control.damping[j] for j in whole_body_keys])

gait_dict = {
    "type":env_cfg.gait.name,
    "frequency":1.2,
    "state_type": env_cfg.gait.state_type.lower(),
    "resample_rate":0,
    "frequency_range":{"max":2,"min":1},
    "stance_when_stop":True 
}
policy_dict = {
    "observation_dim": env_cfg.env.num_observations,
    "action_dim": env_cfg.env.num_actions,
    "encoding_dim": train_cfg.policy.latent_dims,
    "history_length": env_cfg.env.num_history_frames,
    "history_stride": env_cfg.env.history_stride,
    "velocity_command_offset": (0,0),
    "gait": gait_dict
}

actStd: [0.2  0.15 0.5  0.7  0.3  0.2  0.2  0.15 0.5  0.7  0.3  0.2 ]
(49,)
['hip_roll', 'hip_yaw', 'hip_pitch', 'knee_pitch', 'ankle_pitch', 'ankle_roll', 'hip_roll', 'hip_yaw', 'hip_pitch', 'knee_pitch', 'ankle_pitch', 'ankle_roll', 'shoulder_pitch', 'shoulder_roll', 'shoulder_yaw', 'elbow_pitch', 'wrist_yaw', 'wrist_roll', 'wrist_pitch', 'shoulder_pitch', 'shoulder_roll', 'shoulder_yaw', 'elbow_pitch', 'wrist_yaw', 'wrist_roll', 'wrist_pitch', 'waist_yaw', 'waist_pitch', 'waist_roll']


In [10]:
whole_body_keys = []
for j in env_cfg.normalization.whole_body_dof_order:
    if j.startswith("left") or j.startswith("right") :
        whole_body_keys.append(j.rsplit("_",1)[0].split("_",1)[1])
    else:
        whole_body_keys.append(j.rsplit("_",1)[0])
print(whole_body_keys)

['hip_roll', 'hip_yaw', 'hip_pitch', 'knee_pitch', 'ankle_pitch', 'ankle_roll', 'hip_roll', 'hip_yaw', 'hip_pitch', 'knee_pitch', 'ankle_pitch', 'ankle_roll', 'shoulder_pitch', 'shoulder_roll', 'shoulder_yaw', 'elbow_pitch', 'wrist_yaw', 'wrist_roll', 'wrist_pitch', 'shoulder_pitch', 'shoulder_roll', 'shoulder_yaw', 'elbow_pitch', 'wrist_yaw', 'wrist_roll', 'wrist_pitch', 'waist_yaw', 'waist_pitch', 'waist_roll']


In [11]:
if os.path.exists(save_dir):
    for f in os.listdir(save_dir):
        if f.endswith(".pt"):
            os.remove(f"{save_dir}/{f}")
#     os.rmdir(save_dir)
else:
    os.makedirs(save_dir)
en_sc.save(f"{save_dir}/encoder.pt")
ac_sc.save(f"{save_dir}/backbone.pt")
p1_sc.save(f"{save_dir}/policy.pt")

In [12]:
np.savetxt(f"{save_dir}/obMean.csv", obsMean, delimiter=",", fmt="%.1f")
np.savetxt(f"{save_dir}/obStd.csv", obsStd, delimiter=",", fmt="%.2f")
np.savetxt(f"{save_dir}/acMean.csv", actMean, delimiter=",", fmt="%.3f")
np.savetxt(f"{save_dir}/acMargin.csv", acMargin, delimiter=",", fmt="%.1f")
np.savetxt(f"{save_dir}/acStd.csv", actStd, delimiter=",", fmt="%.2f")
np.savetxt(f"{save_dir}/jointPgain.csv", pGain, delimiter=",", fmt="%d")
np.savetxt(f"{save_dir}/jointDgain.csv", dGain, delimiter=",", fmt="%.1f")
with open(f"{save_dir}/policy_config.yaml","w",encoding="UTF-8") as f:
    dump(policy_dict, f, RoundTripDumper)

In [13]:
enc_reload = torch.jit.load(f"{save_dir}/encoder.pt")
bcb_reload = torch.jit.load(f"{save_dir}/backbone.pt")
pol_reload = torch.jit.load(f"{save_dir}/policy.pt")

In [14]:
obs_test = torch.rand(48,450,env_cfg.env.num_observations_history).float()
z_ld = enc_reload(obs_test)
obz_ld = torch.cat([obs_test[..., -env_cfg.env.num_observations:],z_ld], dim=-1)
action_ld = bcb_reload(obz_ld)

In [15]:
act_as_1 = ep(obs_test)

z_sc = en_sc(obs_test)
obz_sc = torch.cat([obs_test[..., -env_cfg.env.num_observations:],z_sc], dim=-1)
action_sc = ac_sc(obz_sc)

act_exp_as_1 = pol_reload(obs_test)
obs_gpu = obs_test.to(ppo_runner.device)
ppo_runner.alg.actor_critic.to(ppo_runner.device)
act_train = ppo_runner.alg.actor_critic(obs_gpu[..., -env_cfg.env.num_observations:], obs_gpu)

print("Train AC(VAE):   ", act_train.shape)
print("Nb Export Class: ", act_as_1.shape)
print("Nb jit Scripts:  ", action_sc.shape)
print("Gym Export Class:", act_exp_as_1.shape)
print("Reloaded jit:    ", action_ld.shape)

Train AC(VAE):    torch.Size([48, 450, 12])
Nb Export Class:  torch.Size([48, 450, 12])
Nb jit Scripts:   torch.Size([48, 450, 12])
Gym Export Class: torch.Size([48, 450, 12])
Reloaded jit:     torch.Size([48, 450, 12])


In [16]:
set_len = 1*int(env.max_episode_length)
obs_np = np.zeros((set_len, env.num_envs, env.num_obs), dtype=float)
act_np = np.zeros((set_len, env.num_envs, env.num_actions), dtype=float)
vel_np = np.zeros((set_len, env.num_envs, 3), dtype=float)
est_np = np.zeros((set_len, env.num_envs, 3), dtype=float)
torq_np = np.zeros((set_len, env.num_envs, env.num_actions), dtype=float)
rew_np = np.zeros((set_len, env.num_envs, len(env.reward_names)), dtype=float)
rew_Sum = np.zeros((set_len, env.num_envs), dtype=float)
gait_rwd_weight_np = np.zeros((set_len, env.num_envs, 4), dtype=float)
cnct_force_np = np.zeros((set_len, env.num_envs,2), dtype=float)
for i in range(1):
    obs,_,obs_h,_ = env.reset()
    print(i, obs)

0 tensor([[ 8.7187e-02,  1.1705e-01,  5.8225e-02,  5.1190e-03, -4.4355e-02,
         -1.0169e+00,  1.5428e+00,  3.3511e-01,  6.6667e-02,  3.0466e-02,
          1.3646e-03,  2.3607e-01, -1.3866e-01, -6.9986e-02,  9.5497e-02,
          1.1660e-01, -8.9132e-04,  9.4597e-02, -2.8511e-01, -8.1305e-02,
          1.5674e-03,  3.2147e-02, -1.7684e-02, -2.0521e-01,  2.9670e-01,
         -9.9945e-02, -5.1531e-01,  4.1567e-02, -8.8794e-02, -6.2850e-01,
          8.7872e-01, -3.6736e-01, -7.8002e-02,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         -1.0000e+00, -1.8370e-16, -1.0000e+00, -1.8370e-16],
        [ 1.0417e-01, -1.9871e-01,  1.5763e-02,  3.3754e-02, -4.2867e-02,
         -9.6821e-01,  2.5715e-01, -1.8232e-01, -6.6667e-02,  1.4535e-01,
         -8.0964e-02, -3.4938e-01, -3.1961e-01,  7.3741e-01, -7.5924e-02,
          3.9694e-02, -3.1744e-02, -1.0808e-01, 

In [17]:
for i in range(3):
    _ = env.reset()

In [18]:
for i in range(obs_np.shape[0]):
    if i <1000:
        env.cmd_w[:,0] = 0.6
        env.cmd_w[:,1] = 0.0
        env.commands[:,0] = 0.6
        env.commands[:,1] = 0.0
        env.commands[:,3] = 0.0
    elif i >=1000:
        env.cmd_w[:,0] = 0.
        env.cmd_w[:,1] = 0.0
        env.commands[:,3] = 0.0
    
#     forward = quat_apply(env.base_quat, env.forward_vec)
#     heading = torch.atan2(forward[:, 1], forward[:, 0])  # yaw
#     env.commands[:,3] = heading
#     print(env.commands[:,:2])
    
    actions =ep(obs_h.cpu())
#     latent = enc_reload(obs_h.cpu())
    (obs, _, obs_h, est_dict,_), rews, dones, infos = env.step(actions.detach())
    torque = env.get_torque()[:, env.action_dof_indices]
    act_np[i] = actions.detach().cpu().numpy()
    obs_np[i] = obs.detach().cpu().numpy()
    vel_np[i] = env.body_vel_buf.detach().cpu().numpy() / env_cfg.normalization.obs_scales.lin_vel
#     est_np[i] = latent[:,:3].detach().cpu().numpy()/ env_cfg.normalization.obs_scales.lin_vel
    torq_np[i] = torque.detach().cpu().numpy()
#     rew_Sum[i] = rews.detach().cpu().numpy()
#     gait_rwd_weight_np[i] = np.hstack((env.force_reward_weight.detach().cpu().numpy(), 
#                                       env.speed_reward_weight.detach().cpu().numpy()))
#     cnct_force_np[i] = torch.norm(env.contact_forces[:, env.feet_indices, :], dim=-1).detach().cpu().numpy()
    
#     for j in range(len(env.reward_names)):
#         name = env.reward_names[j]
#         rew_np[i,:,j] = env.step_reward[name].detach().cpu().numpy()
#     if i%100==99:
#         print(i, obs, "\n", obs_h[0,:env_cfg.env.num_observations])

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline

In [20]:
%matplotlib auto
%matplotlib auto

Using matplotlib backend: <object object at 0x7fd5fa337bb0>
Using matplotlib backend: TkAgg


In [21]:
for i in range(env.num_envs):
    plt.figure()
    plt.plot(torq_np[:,i,10],alpha=0.5,label="AnkYL")
    plt.plot(torq_np[:,i,4],alpha=0.5,label="AnkYR")
    plt.legend()
    plt.title(i)
    plt.show()

In [22]:
for i in range(4):
    print("RMS(L-R): ",
         np.sqrt(np.mean(torq_np[:,i,4]**2)),
         np.sqrt(np.mean(torq_np[:,i,10]**2)),)

RMS(L-R):  7.10736513442322 5.872767717676261
RMS(L-R):  7.63863057157576 8.94770130908595
RMS(L-R):  7.8863375559136655 8.846982103066928
RMS(L-R):  5.969689476104572 7.172066014213438


In [23]:
for i in range(4):
    print("RMS(L-R): ",
         np.mean(np.abs(torq_np[:,i,4]*obs_np[:,i,21+4])),
         np.mean(np.abs(torq_np[:,i,10]*obs_np[:,i,21+9])))

RMS(L-R):  0.6863777530795896 0.6790592905175996
RMS(L-R):  0.5400379649035133 0.894078147941529
RMS(L-R):  0.8634676771551973 1.3175033047006877
RMS(L-R):  0.567492575988507 0.7849166371593824


In [24]:
fig,axs = plt.subplots(4,3)
for i in range(11):
    ax = axs[i//3][i%3]
    name = env.reward_names[i]
    ax.plot(rew_np[:,0,i])
    ax.set_title(name)
# axs[3,2].plot(np.sum(rew_np[:,0], axis=-1))
# axs[3,2].set_title("rew_Sum")
plt.figure()
plt.plot(np.sum(rew_np[:,0], axis=-1))
plt.title("rew_Sum")
plt.show()

IndexError: list index out of range

In [ ]:
plt.figure()
for i in range(4,7):
    name = env.reward_names[i]
    plt.plot(rew_np[:,5,i], label=name)
    plt.legend()

In [ ]:
title_list = ['leg_R', 'leg_L', 'arm_R', 'arm_L']
end = 2000
plt.figure()
for i in range(env.cfg.gait.num_legs):
    plt.plot(obs_np[:end, 0, -8+i*2], label=title_list[i])
plt.legend()
plt.title("gait signal")

plt.figure()
for i in range(env.cfg.gait.num_legs):
    plt.plot(gait_rwd_weight_np[:end, 0, i], label=title_list[i])
plt.legend()
plt.title("eFrc_rwd_weight")

plt.figure()
for i in range(env.cfg.gait.num_legs):
    plt.plot(gait_rwd_weight_np[:end, 0, i+4], label=title_list[i])
plt.legend()
plt.title("eVel_rwd_weight")

In [ ]:
plt.figure()
for i in range(env.cfg.gait.num_legs):
    plt.plot(cnct_force_np[:end, 0, i], label=title_list[i])
plt.legend()
plt.title("cnct_force")

In [ ]:
print(env.gait_manager.get_swap_offset().astype(int))

In [ ]:
plt.figure()
plt.plot(obs_np[:,0,6]/2,label="command")
plt.plot(vel_np[:,0,0],label="vel")
plt.legend()
plt.show()

In [ ]:
fig,axs = plt.subplots(4,4,figsize=(16,8),sharex="all")
for i in range(16):
    ax = axs[i//4][i%4]
    ax.plot(obs_np[:,i,6]/2,label="command")
    ax.plot(vel_np[:,i,0],label="vel")
#     ax.plot(est_np[:,i,0],alpha=0.5)
axs[0][0].legend()
plt.show()

In [ ]:
fig,axs = plt.subplots(3,4,figsize=(16,8),sharex="all")
for i in range(12):
    ax = axs[i//4][i%4]
    ax.plot(obs_np[:,0,9+i],alpha=0.5,label="obs")
    ax.plot(act_np[:,0,i],alpha=0.5,label="act")
axs[0][0].legend()
plt.show()

In [ ]:
fig,axs = plt.subplots(1,1,figsize=(16,8),sharex="all")
for i in range(1):
    ax = axs #[i//8][i%8]
#     ax.plot(obs_np[:,i,6]/2,alpha=0.5)
    ax.plot(vel_np[:,i,0],alpha=0.5)
    ax.plot(est_np[:,i,0],alpha=0.5)
plt.show()

fig,axs = plt.subplots(1,1,figsize=(16,8),sharex="all")
axs.plot(np.mean((vel_np[:,:,0]-est_np[:,:,0])**2,axis=0))
plt.show()

In [ ]:
print(obs_np.shape)
fig,axs = plt.subplots(2,1,figsize=(16,8),sharex="all")
for i in range(4):
    ax = axs[0]
    ax.plot(obs_np[:,1,45+i])
for i in range(3):
    ax = axs[1]
    ax.plot(obs_np[:,1,6+i],label=f"{i}")
ax.legend()
plt.show()

In [ ]:
v_err = obs_np[...,6]/2 - vel_np[...,0]
[np.sqrt(np.mean(v_err**2)), np.mean(v_err), np.max(v_err), np.min(v_err), np.percentile(v_err,75), np.percentile(v_err,25)]

In [ ]:
v_err.flatten().shape

In [ ]:
datasets = {"obSet":obs_np, "acSet":act_np, "velSet": vel_np, "estSet":est_np}
for k,d in datasets.items():
    if f"{k}.csv" in os.listdir(save_dir):
        os.remove(f"{save_dir}/{k}.npy")
    np.save(f"{save_dir}/{k}.npy", d)

In [ ]:
# obSet = np.loadtxt(f"{save_dir}/obSet.csv", delimiter=",")obSet = np.loadtxt(f"{save_dir}/obSet.csv", delimiter=",")
# acSet = np.loadtxt(f"{save_dir}/acSet.csv", delimiter=",")

In [ ]:
# obs_test[:,:] = 0
# for i in range(obSet.shape[0]):
#     obs_test = torch.cat([obs_test[:,ep.ob_dims:], torch.from_numpy(obSet[i]).unsqueeze(0).float()],dim=-1)
#     obs_gpu = copy.deepcopy(obs_test).to(ppo_runner.device)
#     act_as_1 = ep(obs_test)

#     z_sc = en_sc(obs_test)
#     obz_sc = torch.cat([obs_test[:, -env_cfg.env.num_observations:],z_sc], dim=-1)
#     action_sc = ac_sc(obz_sc)

#     act_exp_as_1 = p1_sc(obs_test)

#     ppo_runner.alg.actor_critic.to(ppo_runner.device)
#     act_train = policy(obs_gpu[:, -env_cfg.env.num_observations:], obs_gpu)
    
#     z_ld = enc_reload(obs_test)
#     obz_ld = torch.cat([obs_test[:, -env_cfg.env.num_observations:],z_ld], dim=-1)
#     action_ld = bcb_reload(obz_ld)
#     if i%100==99:
#         print("\n",i)
#         print("Nb Export Class: ", act_as_1)
#         print("Nb jit Scripts:  ", action_sc)
#         print("Gym Export Class:", act_exp_as_1)
#         print("Reloaded jit:    ", action_ld)
#         print("Dataset action:  ", torch.from_numpy(acSet[i]))

In [ ]:
a = torch.tensor([1,2,3,4])
b = torch.tensor([5,6,7,8])
c = np.zeros((2,8), dtype=float)
c[0] = np.hstack((a.numpy(),b.numpy()))
print(c)

In [ ]:
horizontal_scale = 0.1  # [m]
vertical_scale = 0.005

tot_rows = 10
tot_cols = 10
height_field_raw = np.zeros((tot_rows , tot_cols), dtype=np.int16)

width = 10
length = 10
slope = 0.3
x = np.arange(0, width)
y = np.arange(0, length)
xx, yy = np.meshgrid(x, y, sparse=True)
xx = xx.reshape(width, 1)
yy = yy.reshape(length, 1)
xx = xx.squeeze(1)
print(xx)
print(yy)
max_height = int(slope * (horizontal_scale / vertical_scale) * width)
height_field_raw[np.arange(width),:] += (max_height * xx / width).astype(height_field_raw.dtype)

print(height_field_raw)

In [ ]:
a = torch.tensor([0,0,1,1,0,1], dtype=torch.int)
print(a)
b = torch.randint(0, 10, (6,))
print(b)
id1 = (a == 0).nonzero()
print(id1)
print(b[id1])
c = a.unsqueeze(1)
print(c)

In [ ]:
b = torch.randint(0, 20, (8,))
print(b)
a = torch.tensor([3,4,1,2,0,1], dtype=torch.int)
print(b[a])